In [1]:
!pip install openpyxl

In [2]:

import datetime
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
DATA_DIR = os.path.join("data")

In [4]:
fp = os.path.join(DATA_DIR, "hydrograph-excel-sheet-tp-cleaned.xlsx")
xl = pd.ExcelFile(fp)
gages = xl.sheet_names
hydro_data = {s: xl.parse(s) for s in gages}

In [5]:
def flatten_sheet(sheet_name: str, src_data: dict):
    src_df = src_data[sheet_name]
    
    # Check lengths of columns, some contain only notes so will be
    # much less than 100 and need to be dropped. Most columns
    # should have 365/366 values but a few are missing and need to be filled.
    col_lengths = {c: sum(src_df[c].notna()) for c in src_df.columns}
    keep_cols = [c for c, l in col_lengths.items() if l > 100]
    
    # Check columns are all in the correct order to combine:
    assert "time" in keep_cols[0].lower()
    correct_order = {"time": "ft", "ft": "discharge", "discharge": "time"}
    for i, col in enumerate(keep_cols[:-1]):
        next_col = keep_cols[i+1]
        for key in correct_order.keys():
            if key in col.lower():
                should_be = correct_order[key]
                assert should_be in next_col.lower(), sheet_name
    
    # Iterate through columns and collect data:
    data_subsets = list()
    for start_col in range(0, len(keep_cols), 3):
        df_columns = keep_cols[start_col: start_col+3]
        subset = src_df[df_columns]
        rename = dict(zip(subset.columns, ["time", "ft", "m3"]))
        subset = subset.rename(columns=rename).dropna(how="all")
        data_subsets.append(subset)
        
    # Combine to a single df:
    final =  pd.concat(data_subsets).reset_index(drop=True)
    final["gage"] = sheet_name
    return final

In [6]:
all_sheets = list()
for sname in gages:
    all_sheets.append(flatten_sheet(sname, hydro_data)) 
df = pd.concat(all_sheets).reset_index(drop=True)

In [7]:
df.shape

(83274, 4)

In [9]:
df.head()

,time,ft,m3,gage
0,1984-10-01,54.0,1.529110,11402000
1,1984-10-02,52.0,1.472476,11402000
2,1984-10-03,49.0,1.387525,11402000
3,1984-10-04,49.0,1.387525,11402000
4,1984-10-05,48.0,1.359209,11402000


In [28]:
# convert column gage to numeric column
df['gage'] = pd.to_numeric(df['gage'])
df['gage'].value_counts()
# Total of 7 gages

11402000    12418
11318500    12418
11266500    12418
11208000    12053
11185500    12053
11189500    11322
11202710    10592
Name: gage, dtype: int64

## Join table between gage lat lon and gage discharge

In [11]:
# 1. read gage lat lon data
gage_loc = pd.read_csv(os.path.join(DATA_DIR,'target_gages.csv'))

In [17]:
# 2. take subset of the data
lat_lon = gage_loc[['site_no','dec_lat_va','dec_long_va']]
lat_lon.columns = ['gage','lat','lon']
lat_lon.head()

,gage,lat,long
0,11185500,35.905504,-118.467586
1,11189500,35.737452,-118.173689
2,11202710,36.161336,-118.709536
3,11208000,36.521890,-118.799265
4,11266500,37.716871,-119.666279


In [29]:
df['gage'].dtype

dtype('int64')

In [33]:
# 3. Join Data
gage_data = df.merge(lat_lon,on = 'gage',how = 'inner')
gage_data.head()

,time,ft,m3,gage,lat,long
0,1984-10-01,54.0,1.529110,11402000,40.002947,-120.954399
1,1984-10-02,52.0,1.472476,11402000,40.002947,-120.954399
2,1984-10-03,49.0,1.387525,11402000,40.002947,-120.954399
3,1984-10-04,49.0,1.387525,11402000,40.002947,-120.954399
4,1984-10-05,48.0,1.359209,11402000,40.002947,-120.954399


In [34]:
# save csv file
gage_data.to_csv('gage_discharge_lat_lon.csv',index=False)